<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze-.iframe.experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from IPython.display import IFrame
NOTEBOOK_REPO = "voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb"
IFrame(src=f'http://localhost:3000/colabasaservice/?insidecolab={NOTEBOOK_REPO}', width=700, height=600)

In [ ]:
!npm install -g http-server
!nohup http-server -p 8000 &

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

In [ ]:
import asyncio
import websockets


async def hello(websocket, path):
    name = await websocket.recv()
    print("< {}".format(name))

    greeting = "Hello {}!".format(name)
    await websocket.send(greeting)
    print("> {}".format(greeting))

start_server = websockets.serve(hello, 'localhost', 8765)
loop = asyncio.get_event_loop()
loop.create_task(start_server())
#asyncio.ensure_future(start_server())
#asyncio.get_event_loop().run_forever()

In [ ]:
!pip install deep-daze --upgrade
!pip install stomp.py imageio-ffmpeg asyncio websockets --upgrade

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 

flatten = itertools.chain.from_iterable

TEXT="A black and white technical illustration of merging a cat and a human through biohacking."

DEFAULT_PARAMS='--lower_bound_cutout 0.5 --image-width=512 --iterations=5000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)



def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)

    headers = { 
        "params": params,
        "text": TEXT
    }
    conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files

    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          filename = os.path.basename(added)
          mimetype, _ = guess_type(filename)
          print("Adding:", added,"type",mimetype)
          
          with open(added,"rb") as media_file:
            added_media = media_file.read()

            base64_encoded_media = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
          
          headers = {
              **headers,         
              "type": mimetype,
              "filename": filename
              }
          conn.send(body=base64_encoded_media, headers=headers, destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))
run(TEXT)
conn.disconnect()



Streaming output truncated to the last 5000 lines.
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.001383.jpg type image/jpeg
loss: -62.72:  55%|█████▌    | 2767/5000 [18:08<14:37,  2.55it/s]
                                             
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.001384.jpg type image/jpeg
loss: -62.36:  55%|█████▌    | 2769/5000 [18:09<14:39,  2.54it/s]
                                             
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.001385.jpg type image/jpeg
loss: -65.06:  55%|█████▌    | 2771/5000 [18:10<14:40,  2.53it/s]
                                             
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.001386.jpg type image/jpeg
loss: -61.81:  55%|█████▌    | 2773/5000 [18:11<14:36,  2.54it/s]
                                             
Adding: ./A_black_and_white_technical_ill

# New section

In [ ]:
%%javascript
// Create the channel object
const channel = new BroadcastChannel('my-channel');

// Set up a listener
channel.onmessage = message => console.log('Received:', message.data);

// Post a message to all subscribers
channel.postMessage('Hello, world');

In [ ]:
conn.disconnect()

In [ ]:
!pip install simple-websocket-server

In [ ]:
from simple_websocket_server import WebSocketServer, WebSocket


class SimpleEcho(WebSocket):
    def handle(self):
        # echo message back to client
        self.send_message(self.data)

    def connected(self):
        print(self.address, 'connected')

    def handle_close(self):
        print(self.address, 'closed')


server = WebSocketServer('', 8000, SimpleEcho)
server.serve_forever()
loop = asyncio.get_event_loop()
loop.create_task(some_async_function())